# 📘 Clase 23: Testing APIs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-03/clase-23-testing-apis.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Usar TestClient para testar APIs
- Escribir tests para endpoints
- Testar autenticacion en APIs
- Mockear dependencias
- Organizar tests de integracion

---

## 📚 TestClient de FastAPI

In [ ]:
# ============================================
# TESTING BASICO CON TESTCLIENT
# ============================================

TESTING_CODE = '''
from fastapi import FastAPI
from fastapi.testclient import TestClient
from pydantic import BaseModel
import pytest

# App a testear
app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

items_db = []

@app.get("/items")
async def list_items():
    return items_db

@app.post("/items", status_code=201)
async def create_item(item: Item):
    items_db.append(item.dict())
    return item

@app.get("/items/{item_id}")
async def get_item(item_id: int):
    if item_id >= len(items_db):
        raise HTTPException(status_code=404, detail="Not found")
    return items_db[item_id]

# Crear TestClient
client = TestClient(app)

# ========== TESTS ==========

def test_read_items():
    response = client.get("/items")
    assert response.status_code == 200
    assert response.json() == []

def test_create_item():
    response = client.post(
        "/items",
        json={"name": "Foo", "price": 50.5}
    )
    assert response.status_code == 201
    data = response.json()
    assert data["name"] == "Foo"
    assert data["price"] == 50.5

def test_read_item():
    # Primero crear item
    client.post("/items", json={"name": "Bar", "price": 100})
    
    response = client.get("/items/0")
    assert response.status_code == 200
    assert response.json()["name"] == "Foo"

def test_read_item_not_found():
    response = client.get("/items/999")
    assert response.status_code == 404
    assert response.json()["detail"] == "Not found"
'''

print(TESTING_CODE)

---

## 🔐 Testing Autenticacion

In [ ]:
# ============================================
# TESTING CON AUTENTICACION
# ============================================

AUTH_TESTING = '''
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from fastapi.testclient import TestClient

app = FastAPI()
security = HTTPBearer()

async def get_token(
    credentials: HTTPAuthorizationCredentials = Depends(security)
):
    if credentials.credentials != "valid-token":
        raise HTTPException(status_code=401, detail="Invalid token")
    return credentials.credentials

@app.get("/protected")
async def protected_route(token: str = Depends(get_token)):
    return {"message": "Access granted", "token": token}

client = TestClient(app)

def test_protected_without_token():
    response = client.get("/protected")
    assert response.status_code == 403

def test_protected_with_invalid_token():
    response = client.get(
        "/protected",
        headers={"Authorization": "Bearer invalid-token"}
    )
    assert response.status_code == 401

def test_protected_with_valid_token():
    response = client.get(
        "/protected",
        headers={"Authorization": "Bearer valid-token"}
    )
    assert response.status_code == 200
    assert response.json()["message"] == "Access granted"
'''

print(AUTH_TESTING)

---

## 🎭 Fixtures para API Testing

In [ ]:
# ============================================
# FIXTURES PARA TESTING DE API
# ============================================

FIXTURES_CODE = '''
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from app.main import app
from app.database import Base, get_db

# Base de datos de prueba
SQLALCHEMY_DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(SQLALCHEMY_DATABASE_URL)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

@pytest.fixture(scope="function")
def db():
    """Crea una BD limpia para cada test."""
    Base.metadata.create_all(bind=engine)
    db = TestingSessionLocal()
    
    yield db
    
    db.close()
    Base.metadata.drop_all(bind=engine)

@pytest.fixture
def client(db):
    """Cliente de test con BD sobrescrita."""
    def override_get_db():
        try:
            yield db
        finally:
            db.close()
    
    app.dependency_overrides[get_db] = override_get_db
    
    with TestClient(app) as c:
        yield c
    
    app.dependency_overrides.clear()

@pytest.fixture
def test_user(client):
    """Crea un usuario de prueba."""
    user_data = {
        "username": "testuser",
        "email": "test@test.com",
        "password": "secret123"
    }
    response = client.post("/users/", json=user_data)
    return response.json()

@pytest.fixture
def auth_headers(client, test_user):
    """Headers con token de autenticacion."""
    response = client.post(
        "/auth/login",
        data={"username": "testuser", "password": "secret123"}
    )
    token = response.json()["access_token"]
    return {"Authorization": f"Bearer {token}"}

# Uso en tests
def test_create_project(client, auth_headers):
    response = client.post(
        "/projects/",
        json={"name": "Test Project"},
        headers=auth_headers
    )
    assert response.status_code == 201
'''

print(FIXTURES_CODE)

---

**¡Testea tus endpoints como un pro! 🧪**